In [1]:
import os, sys
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['PYSPARK_PYTHON'] =  'python3.9'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3.9'
os.environ['HADOOP_USER_NAME']='ssenigov'

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf, StorageLevel, RDD
from pyspark.sql.functions import col

In [ ]:
conf = SparkConf().setAppName('AddrIdenticalCols').setMaster('yarn')

spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [5]:
df = spark.createDataFrame([('Mary Grant', 11), 
                            ('Bob Itkis', 14),
                            ('Bill Drummond', 12),
                            ('Anna Fletcher', 12),
                            ('John Ayrton', 11)], 
                           ['name', 'age'])
df.show()
df.createOrReplaceTempView("tbl")

+-------------+---+
|         name|age|
+-------------+---+
|   Mary Grant| 11|
|    Bob Itkis| 14|
|Bill Drummond| 12|
|Anna Fletcher| 12|
|  John Ayrton| 11|
+-------------+---+



In [16]:
res_sql = spark.sql("""
  select t1.name, t2.name, t1.age
  from tbl as t1 
     cross join tbl as t2
  where t1.age = t2.age
  and t1.name != t2.name
  order by t1.name
""")
res_sql.show()

+-------------+-------------+---+
|         name|         name|age|
+-------------+-------------+---+
|Anna Fletcher|Bill Drummond| 12|
|Bill Drummond|Anna Fletcher| 12|
|  John Ayrton|   Mary Grant| 11|
|   Mary Grant|  John Ayrton| 11|
+-------------+-------------+---+



In [25]:
df1 = df.alias('df1')
df2 = df.alias('df2')
df_res = df1.join(other=df2,how='cross')
df_res = df_res.where((col('df1.age') == col('df2.age')) & ~ (col('df1.name') == col('df2.name')))
df_res.select(col('df1.name'), col('df2.name'), col('df1.age')).orderBy(col('df1.name')).show()

+-------------+-------------+---+
|         name|         name|age|
+-------------+-------------+---+
|Anna Fletcher|Bill Drummond| 12|
|Bill Drummond|Anna Fletcher| 12|
|  John Ayrton|   Mary Grant| 11|
|   Mary Grant|  John Ayrton| 11|
+-------------+-------------+---+



In [ ]:
 spark.stop()